# Setup

In [1]:
import os
from openai import OpenAI

# openai.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(
    api_key='sk-6898b7358b154a109023e50170a26058',
    base_url="https://api.deepseek.com")
# os.environ["HTTP_PROXY"]  = "http://127.0.0.1:7890"
# os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

def llm(prompt, stop=None):
    response = client.chat.completions.create(  # 请求参数配置
      model="deepseek-chat",
      messages=[{"role": "user", "content": prompt}],
      temperature=0,  # 控制AI回复的随机性,0表示固定答案
      max_tokens=100,  # 限制AI回复的长度
      top_p=1,  # 与temperature配合,1表示选择最可能的词
      frequency_penalty=0.0,  # 控制重复用词,0表示允许重复
      presence_penalty=0.0,
      stop=stop  # 指定停止词
    )
    return response.choices[0].message.content  # API返回结果中的content部分

In [2]:
import wikienv, wrappers
import requests  # 处理请求错误、超时等异常

env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [3]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples  # 指示词和例子

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    obs, info = env.reset(idx=idx)
    question = obs
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):  # ReAct循环,最多7次
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, terminated, truncated, info = step(env, action[0].lower() + action[1:])
        done = terminated or truncated
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, terminated, truncated, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [4]:
import random
import time
idxs = list(range(7405))
random.Random(232).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in idxs[:500]:
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

5149 Question: What is the ratio of flow velocity past a boundary to the local speed of sound for a Saab JAS 39 Gripen?


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))